In [34]:
%matplotlib inline
import pandas as pd
import numpy as np
from IPython.core.interactiveshell import InteractiveShell
import random
InteractiveShell.ast_node_interactivity = "all"
import os
from datetime import datetime
import matplotlib.pyplot as plt
#import plotly.express as px
import glob
from tqdm import tqdm

# Using plotly.express
import plotly.express as px

import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import sys
sys.path.append("../")

import warnings
warnings.filterwarnings('ignore')

from prophet import Prophet
from prophet.plot import plot_plotly, plot_components_plotly

In [2]:
url = 'data/australia_viirs_prepared.csv.gz'
aus_fires = pd.read_csv(url,parse_dates=['time'])
aus_fires.shape
aus_fires.head()

(1323312, 10)

,latitude,longitude,time,fire_count,confidence,est_brightness,frp,est_fire_area,loc_count,ranking
0,-29.5,123.9,2019-02-16,315,79,331.6,358982.2,103.9,4,0.694892
1,-31.4,123.7,2019-11-10,680,71,324.5,24134.6,207.1,11,0.667022
2,-32.4,123.5,2020-01-02,474,86,335.9,11099.8,138.2,12,0.601851
3,-27.6,144.3,2013-01-13,566,68,313.9,13568.5,185.4,2,0.599751
4,-26.3,150.7,2013-10-08,551,66,311.2,10181.1,188.4,51,0.589020


In [29]:
aus_fires.iloc[1000:2000]

,latitude,longitude,time,fire_count,confidence,est_brightness,frp,est_fire_area,loc_count,ranking
1000,-13.7,129.9,2012-07-24,2,100,331.0,35.5,1.0,219,0.401484
1001,-25.3,127.5,2021-08-22,2,100,331.7,34.7,1.0,26,0.401484
1002,-21.2,118.1,2021-12-27,2,100,335.6,34.5,1.0,12,0.401484
1003,-19.7,147.3,2019-10-14,2,100,331.8,33.9,1.0,29,0.401483
1004,-33.4,116.9,2017-06-05,2,100,331.1,33.0,1.0,73,0.401483
...,...,...,...,...,...,...,...,...,...,...
1995,-14.7,130.1,2013-05-29,2,100,333.4,20.0,0.5,137,0.400993
1996,-16.2,127.4,2021-05-29,2,100,336.1,20.0,0.5,65,0.400993
1997,-16.5,127.0,2012-04-18,2,100,334.3,20.0,0.5,61,0.400993
1998,-36.7,143.9,2013-04-08,2,100,335.1,20.0,0.5,26,0.400993


In [3]:
def filterLocation(aus_fires,latitude,longitude):
    filtered = aus_fires[(aus_fires.latitude == latitude)&(aus_fires.longitude == longitude)]
    return filtered

In [44]:
lat = -36.7
lng = 143.9
filtered = filterLocation(aus_fires,lat,lng)

In [45]:
df = filtered.filter(['time','ranking'])
df.rename({'time': 'ds'}, axis=1, inplace=True)
df.rename({'ranking': 'y'}, axis=1, inplace=True)
df

,ds,y
1998,2013-04-08,0.400993
5658,2018-04-08,0.400489
14754,2012-11-20,0.349426
65399,2017-04-13,0.295778
100781,2013-04-07,0.283173
110397,2014-04-06,0.281703
339484,2013-04-09,0.243235
499778,2012-04-17,0.241443
515389,2021-12-30,0.241353
534972,2014-05-07,0.241277


### Prophet with monthly data

In [46]:
# RESAMPLE DATA FROM DAILY TO MONTHLY
df_m = df.copy()
df_m.set_index('ds', inplace=True)
df_m.index = pd.to_datetime(df_m.index)
df_m = df_m.resample('1M').mean()

In [47]:
df_m['y'] = df_m['y'].fillna(0)

In [48]:
df_m

,y
ds,
2012-04-30,0.240897
2012-05-31,0.000000
2012-06-30,0.000000
2012-07-31,0.000000
2012-08-31,0.000000
...,...
2021-08-31,0.000000
2021-09-30,0.000000
2021-10-31,0.000000


In [49]:
df_m.reset_index('ds', inplace=True)

In [50]:
df_m

,ds,y
0,2012-04-30,0.240897
1,2012-05-31,0.000000
2,2012-06-30,0.000000
3,2012-07-31,0.000000
4,2012-08-31,0.000000
...,...,...
112,2021-08-31,0.000000
113,2021-09-30,0.000000
114,2021-10-31,0.000000
115,2021-11-30,0.000000


In [51]:
m = Prophet()
m.fit(df_m)

INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


Initial log joint probability = -26.3707


    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       138.533   5.21606e-07       93.5739      0.9173     0.09173      127   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     122       138.533   9.73934e-09       109.538      0.2309           1      155   
Optimization terminated normally: 
  Convergence detected: absolute parameter change was below tolerance


In [52]:
future = m.make_future_dataframe(periods=24,freq = "m")
future.tail()

,ds
136,2023-08-31
137,2023-09-30
138,2023-10-31
139,2023-11-30
140,2023-12-31


In [53]:
forecast = m.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

,ds,yhat,yhat_lower,yhat_upper
136,2023-08-31,-0.021415,-0.100096,0.062025
137,2023-09-30,-0.021959,-0.103584,0.061372
138,2023-10-31,0.003242,-0.087056,0.086332
139,2023-11-30,0.009677,-0.068925,0.099286
140,2023-12-31,-0.000140,-0.079128,0.079290


In [54]:
plot_plotly(m, forecast)
plot_components_plotly(m, forecast)

 ### Prophet with weekly data

In [17]:
# RESAMPLE DATA FROM DAILY TO WEEKLY
df_w = df.copy()
df_w.set_index('ds', inplace=True)
df_w.index = pd.to_datetime(df_w.index)
df_w = df_w.resample('1W').mean()

In [18]:
df_w

,y
ds,
2012-05-06,0.261288
2012-05-13,NaN
2012-05-20,0.240987
2012-05-27,NaN
2012-06-03,NaN
...,...
2021-04-18,0.240618
2021-04-25,NaN
2021-05-02,NaN


In [19]:
df_w['y'] = df_w['y'].fillna(0)

In [20]:
df_w.reset_index('ds', inplace=True)

In [21]:
df_w

,ds,y
0,2012-05-06,0.261288
1,2012-05-13,0.000000
2,2012-05-20,0.240987
3,2012-05-27,0.000000
4,2012-06-03,0.000000
...,...,...
467,2021-04-18,0.240618
468,2021-04-25,0.000000
469,2021-05-02,0.000000
470,2021-05-09,0.000000


In [22]:
w = Prophet()
w.fit(df_w)

INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


Initial log joint probability = -133.058


    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99        563.17   1.06981e-05       94.5599      0.7093      0.7093      128   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     159       563.173   2.13814e-08       99.9772      0.2766           1      218   
Optimization terminated normally: 
  Convergence detected: relative gradient magnitude is below tolerance


In [23]:
future = m.make_future_dataframe(periods=10,freq = "w")
future.tail()

,ds
114,2021-07-11
115,2021-07-18
116,2021-07-25
117,2021-08-01
118,2021-08-08


In [24]:
forecast = w.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

,ds,yhat,yhat_lower,yhat_upper
114,2021-07-11,0.014626,-0.058432,0.090039
115,2021-07-18,0.009699,-0.066886,0.081598
116,2021-07-25,0.005349,-0.073113,0.084355
117,2021-08-01,0.004875,-0.070848,0.079235
118,2021-08-08,0.005576,-0.073496,0.079893


In [25]:
plot_plotly(w, forecast)
plot_components_plotly(w, forecast)